Vacuum cleaner AI.

This program is a vacuum cleaner AI that via a search algorithm navigates through a room with blocks where it can't move. Instead of placing dust and dirt, we are instead requiring that the vacuum passes every square that it can with respect to its starting position and the blocks in the room.

Written in collaboration with Danai Deligeorgaki. 

In [68]:
import numpy as np
from sklearn.utils import shuffle

n = 7 # number of rows
m = 7 # number of columns
k = 10 # number of obstacles
A=np.zeros((n,m)) # The initialized matrix describing the room
obst = (n*m)*10 # value we assign to the entries of A corresponding to each obstacle

Next we initialize the position of the obstacles and then the starting position of the vacuum. 

In [69]:
counter=0 # count number of obstacles that have been placed
while counter<k:
    r = np.random.randint(n) #pick a random row of A
    c = np.random.randint(m) #pick a random column of A
    if A[r][c]==0: #check that there is not another obstacle assigned to the entry of A that was randomly chosen
        A[r][c]= obst #update the chosen entry of A with the obstacle value
        counter=counter+1

Ainitial=np.array(A,copy=True)
vac_r=-1
vac_c=-1 #the initial position of the vacuum will be later set to be (var_r,var_c)

placed = False # placed is put to True when the initial position of the vacuum has been determined
while placed == False:
    r = np.random.randint(n) # pick a random row of A
    c = np.random.randint(m) # pick a random column of A
    if A[r][c]==0: # check that there is no obstacle assigned to the entry of A that was randomly chosen
        placed = True
        vac_r=r
        vac_c=c # set the initial position of the vacuum to (var_r,var_c)
        A[r][c]=np.intc(1) # assing value 1 to the initial position of the vacuum

In [70]:
#print(Ainitial)
print(A)

[[  0.   0.   0.   0.   0.   0.   0.]
 [  0. 490.   0.   0.   0.   0.   0.]
 [  0.   0. 490.   0.   0.   0. 490.]
 [  0.   0. 490.   0. 490. 490.   0.]
 [  0.   1.   0.   0. 490. 490.   0.]
 [  0.   0. 490.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. 490.]]


Here we implement the AI algorithm.

In [71]:
def AI_alg(M,deadEnd): # Input matrix with vaccum history the position of dead-end.
    # Idea: Where M has value 0<x<obst put a -1 except put a 1 at the dead-end. 
    # (i) At the dead-end, check the surrounding 4 boxes, put a 2 where there is a -1.
    # Repeat (i) for every entry of M of value 2, and put value 3 where there is a -1 instead. 
    # Repeat (i) for every entry of M of value 3, and put value 4 where there is a -1 instead and so on...
    # Key idea: When the loop is done, the value of each square of M that is not an obstacle and that is reachable by 
    # the dead-end corresponds to the least amount of steps needed to get there. 
    # If one of the 4 surrounding boxes is ever of value 0, break the loop. We have found a closest zero. 
    # Set the closest zero as destination and set variable count= its corresponding value.
    # If no zero is ever found, break the loop, and set destination to the starting point, with count= the starting
    # point's corresponding value.
    # Back-track by going to a square of one lower value until we reach the dead-end. This gives us a fastest path to
    # The closest zero. 
    Mstar=np.array(M,copy=True)
    for i in range(n):
        for j in range(m):
            if Mstar[i,j]>=1 and Mstar[i,j]<obst:
                Mstar[i,j]=-1

    Mstar[deadEnd[0],deadEnd[1]]=1 
    done=False
    curVal=1
    dest_r=-1
    dest_c=-1
    while done==False:
        changeMade=False # If change is made to Mstar in the for loops below, set to true 
        for i in range(n):
            for j in range(m):
                if done==False and Mstar[i,j]==curVal:
                    if done==False and i-1>=0:
                        if Mstar[i-1,j]==-1: 
                            Mstar[i-1,j]=curVal+1
                            changeMade=True
                        if Mstar[i-1,j]==0:
                            changeMade=True
                            done=True
                            dest_r=i
                            dest_c=j
                    if done==False and i+1<n:
                        if Mstar[i+1,j]==-1: 
                            Mstar[i+1,j]=curVal+1
                            changeMade=True
                        if Mstar[i+1,j]==0:
                            changeMade=True
                            done=True
                            dest_r=i
                            dest_c=j
                    if done==False and j-1>=0:
                        if Mstar[i,j-1]==-1: 
                            Mstar[i,j-1]=curVal+1
                            changeMade=True
                        if Mstar[i,j-1]==0:
                            changeMade=True
                            done=True
                            dest_r=i
                            dest_c=j
                    if done==False and j+1<m:
                        if Mstar[i,j+1]==-1: 
                            Mstar[i,j+1]=curVal+1
                            changeMade=True
                        if Mstar[i,j+1]==0:
                            done=True
                            changeMade=True
                            dest_r=i
                            dest_c=j
        curVal=curVal+1
        if changeMade==False: #Return to start
            dest_r=vac_r
            dest_c=vac_c
            done=True

    arr_backtrack=[[dest_r,dest_c]]
    count=Mstar[dest_r,dest_c] # curVal
    while count>1:
        bt=arr_backtrack[0]
        if bt[0]-1>=0 and Mstar[bt[0]-1,bt[1]]==count-1: 
            arr_backtrack=[[bt[0]-1,bt[1]]]+arr_backtrack
        elif bt[0]+1<n and Mstar[bt[0]+1,bt[1]]==count-1: 
            arr_backtrack=[[bt[0]+1,bt[1]]]+arr_backtrack
        elif bt[1]-1>=0 and Mstar[bt[0],bt[1]-1]==count-1: 
            arr_backtrack=[[bt[0],bt[1]-1]]+arr_backtrack
        elif bt[1]+1<m and Mstar[bt[0],bt[1]+1]==count-1: 
            arr_backtrack=[[bt[0],bt[1]+1]]+arr_backtrack
        count=count-1

    return arr_backtrack

Next we write the code for running the vacuum. We introduce the variables of the current position and keep track of each step we are taking. At each step we are randomizing the direction we would like to go, and check if there is a zero there. If there is no zero, then we start to backtrack, unless we are at the starting position, then we are finished.

In [72]:
cur_pos_r=vac_r # (cur_pos_r,cur_pos_c) denotes the current position of the vacuum, and varies as the algorithm runs.
cur_pos_c=vac_c 

step=1 # Placing the vacuum was the first step of the algorithm. 
       # As the vacuum moves to neighbouring blocks that have not been visited before, the step increases by 1. 

done=False # The algorithm is completed when done==True. 
while done == False:
    arr=shuffle(np.arange(4)) # Randomly prioritize the direction that the vacuum will attempt to move to:
                              # 0 is up, 1 is down, 2 is left, 3 is right.
    dire=-1 # We initialize the direction as -1    
    counter=0
   
    while dire==-1 and counter <4:
        if arr[counter]==0 and cur_pos_r>0: # if the preferred direction is "up" and the vacuum can move "up" in A
            if A[cur_pos_r-1][cur_pos_c]==0: # if the entry of A "up" compared to the vacuum has value "0"
                dire=0 # means that the direction the vacuum will take after the while loop is "up"
                step=step+1
                A[cur_pos_r-1][cur_pos_c]=step # assign the value of "step" to the entry of the table that is "up"
        elif arr[counter]==1 and cur_pos_r<n-1:
            if A[cur_pos_r+1][cur_pos_c]==0: 
                dire=1
                step=step+1
                A[cur_pos_r+1][cur_pos_c]=step
        elif arr[counter]==2 and cur_pos_c>0:
            if A[cur_pos_r][cur_pos_c-1]==0:
                dire=2
                step=step+1
                A[cur_pos_r][cur_pos_c-1]=step
        elif arr[counter]==3 and cur_pos_c<m-1:
            if A[cur_pos_r][cur_pos_c+1]==0:
                dire=3
                step=step+1
                A[cur_pos_r][cur_pos_c+1]=step
        counter=counter+1
   
    if dire==-1: # if there is no block with entry "0" that the vacuum can reach with one move, then dire==-1
        if cur_pos_r==vac_r and cur_pos_c==vac_c: # if, additionally, the vacuum is located in its initial position
            done = True # the algorithm ends
        else: # otherwise the vacuum should backtrack
            arr_bt=AI_alg(A,[cur_pos_r,cur_pos_c])
            for i in range(1,len(arr_bt)):
                cur_pos_r=arr_bt[i][0]
                cur_pos_c=arr_bt[i][1]
        
         
    if dire!=-1 and done==False: # if we are not done, then here we update the position of the vacuum cleaner 
        if dire==0:
            cur_pos_r=cur_pos_r-1
            cur_pos_c=cur_pos_c
        elif dire==1:
            cur_pos_r=cur_pos_r+1
            cur_pos_c=cur_pos_c
        elif dire==2:
            cur_pos_r=cur_pos_r
            cur_pos_c=cur_pos_c-1
        elif dire==3:
            cur_pos_r=cur_pos_r
            cur_pos_c=cur_pos_c+1 

Below we print the matrix A to see that the way the vacuum moves makes sense with respect to the code above

In [73]:
A

array([[  8.,   9.,  10.,  11.,  12.,  15.,  16.],
       [  7., 490.,  22.,  21.,  13.,  14.,  17.],
       [  6.,   5., 490.,  20.,  19.,  18., 490.],
       [  3.,   4., 490.,  23., 490., 490.,  38.],
       [  2.,   1.,  34.,  24., 490., 490.,  37.],
       [ 32.,  31., 490.,  25.,  26.,  35.,  36.],
       [ 33.,  30.,  29.,  28.,  27.,  39., 490.]])